<h1 style="line-height:.1;"><center>Tugas Akhir Kecerdasan Buatan TIF-A SA</center></h1>
<h1 style="line-height:.5;"><center>Catatan Pengerjaan Program</center></h1>

---
Dimas Tri Mustakim (205150200111049)

# Kode Lingkungan / Environment

Peta lingkungan dibuat berdasarkan kode program dari salah satu lingkungan yang tersedia di library OpenAI Gym yang bernama Taxi. Disini kami menggunakan kode program tersebut dan memodifikasinya untuk memenuhi topik yang kami pilih yaitu tentang robot pengangkut barang. Berikut beberapa hal yang saya ubah.

1. Merubah ukuran lingkungan menjadi 10x10 dari semula 4x4
2. Merubah letak goal
3. Merubah model reward
4. Sedikit modifikasi pewarnaan lingkungan saat render
5. Merubah beberapa parameter
6. Membuat method step() untuk assert nilai masukan untuk memastikan nilai masukan 
7. Merubah nama variable untuk menggambarkan lingkungan robot pengangkut barang

Link source dari lingkungan kami ada disini : [Taxi milik OpenAI Gym](https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py).

Source code tersebut dirilis secara open source dengan lisensi MIT dan dapat dilihat di link berikut ini
* https://github.com/openai/gym/blob/master/LICENSE.md

##### Highlight perubahan kode

In [3]:
# Peta lingkungan/environment
# Karakter '|' dan ':' dapat direplace satu sama lain untuk memodifikasi

MAP = [
    "+-------------------+",
    "|R| | : | : | : : :G|",
    "| | | | : : | : : : |",
    "| | : | : : | : | : |",
    "| | : | : : | : | : |",
    "| | : | : : | : | : |",
    "| | : | : |B| : | : |",
    "| : : | : : : : | : |",
    "| | : | : : : : | : |",
    "| | : | : : : : | : |",
    "|Y| : : : : | : : : |",
    "+-------------------+",
]

In [12]:
class DeliveryRobotEnv(discrete.DiscreteEnv):
    
    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self):
        self.desc = np.asarray(MAP, dtype='c')

        self.locs = locs = [(0, 0), (0, 9), (9, 0), (5, 5)]

        num_states = 2000  # 10 * 10 * 5 * 4
        num_rows = 10
        num_columns = 10
        max_row = num_rows - 1
        max_col = num_columns - 1
        initial_state_distrib = np.zeros(num_states)
        num_actions = 6
        P = {state: {action: [] for action in range(num_actions)} for state in range(num_states)}

        for row in range(num_rows):
            for col in range(num_columns):
                for goods_idx in range(len(locs) + 1):
                    for dest_idx in range(len(locs)):
                        state = self.encode(row, col, goods_idx, dest_idx)
                        if goods_idx < 4 and goods_idx != dest_idx:
                            initial_state_distrib[state] += 1
                        for action in range(num_actions):
                            new_row, new_col, new_goods_idx = row, col, goods_idx
                            reward = -1 
                            done = False
                            robot_loc = (row, col)

                            if action == 0:
                                new_row = min(row + 1, max_row)
                            elif action == 1:
                                new_row = max(row - 1, 0)
                            if action == 2 and self.desc[1 + row, 2 * col + 2] == b":":
                                new_col = min(col + 1, max_col)
                            elif action == 3 and self.desc[1 + row, 2 * col] == b":":
                                new_col = max(col - 1, 0)
                            elif action == 4:  # pickup
                                if goods_idx < 4 and robot_loc == locs[goods_idx]:
                                    new_goods_idx = 4
                                else:  # goods not at location
                                    reward = -10
                            elif action == 5:  # dropoff
                                if (robot_loc == locs[dest_idx]) and goods_idx == 4:
                                    new_goods_idx = dest_idx
                                    done = True
                                    reward = 30
                                elif (robot_loc in locs) and goods_idx == 4:
                                    new_goods_idx = locs.index(robot_loc)
                                else:  # dropoff at wrong location
                                    reward = -10
                            new_state = self.encode(new_row, new_col, new_goods_idx, dest_idx)
                            P[state][action].append((1.0, new_state, reward, done))

        initial_state_distrib /= initial_state_distrib.sum()
        discrete.DiscreteEnv.__init__(self, num_states, num_actions, P, initial_state_distrib)

    def encode(self, robot_row, robot_col, goods_loc, dest_idx):
        """
        Berfungsi untuk menencode keadaan lingkungan yang berupa (robot_row, robot_col, lokasi barang, tujuan)
        ke dalam suatu angka diskrit untuk merepresentasikan keadaan lingkungan
        
        4 data keadaan lingkungan tersebut akan di encode ke dalam nilai diskrit di antara 0 sampai 2000
        
        """
        i = robot_row
        i *= 10
        i += robot_col
        i *= 5
        i += goods_loc
        i *= 4
        i += dest_idx
        return i

    def decode(self, i):
        """
        Untuk men-decode angka representasi state kembali ke bentuk (robot_row, robot_col, lokasi_barang, tujuan)
        """
        out = []
        out.append(i % 4)
        i = i // 4
        out.append(i % 5)
        i = i // 5
        out.append(i % 10)
        i = i // 10
        out.append(i)
        assert 0 <= i < 10
        return reversed(out)

    def step(self, a):
        assert 0 <= a < 2000
        return super().step(a)

    def render(self, mode='human'):
        outfile = StringIO() if mode == 'ansi' else sys.stdout

        out = self.desc.copy().tolist()
        out = [[c.decode('utf-8') for c in line] for line in out]
        robot_row, robot_col, goods_idx, dest_idx = self.decode(self.s)

        # Coloring barrier ( '|', '+', '-' )
        for a in out:
            for b in range(len(a)):
                if a[b] == '|' or a[b] == '+' or a[b] == '-':
                    a[b] = utils.colorize(a[b], 'cyan')

        def ul(x):
            return "_" if x == " " else x

        if goods_idx < 4:
            out[1 + robot_row][2 * robot_col + 1] = utils.colorize(
                out[1 + robot_row][2 * robot_col + 1], 'yellow', highlight=True)
            pi, pj = self.locs[goods_idx]
            out[1 + pi][2 * pj + 1] = utils.colorize(out[1 + pi][2 * pj + 1], 'blue', bold=True)
        else:  # passenger in taxi
            out[1 + robot_row][2 * robot_col + 1] = utils.colorize(
                ul(out[1 + robot_row][2 * robot_col + 1]), 'green', highlight=True)

        di, dj = self.locs[dest_idx]
        out[1 + di][2 * dj + 1] = utils.colorize(out[1 + di][2 * dj + 1], 'magenta')
        outfile.write("\n".join(["".join(row) for row in out]) + "\n")
        if self.lastaction is not None:
            outfile.write("  ({})\n".format(
                ["Selatan", "Utara", "Timur", "Barat", "Ambil Barang", "Taruh Barang"][self.lastaction])
            )
            
        else:
            outfile.write("\n")

        if mode != 'human':
            with closing(outfile):
                return outfile.getvalue()


# Penggunaan lingkungan
looping episode, cara interaksi, penggunaan method dan berbagai parameter mengikuti standar yang ditetapkan oleh OpenAI Gym kepada lingkungan mereka. Jadi disini kode untuk penggunaan lingkungan kami ini mengikuti petunjuk yang telah ditetapkan pada dokumentasi library milik mereka yang bisa diakses **[disini](https://gym.openai.com/docs/)**. 

# Kode Q-Learning

Kode untuk Q-Learning (q-table, parameter yang digunakan, rumus dan kode untuk update q-value) saya pelajari dari beberapa sumber di internet (youtube, blog, medium) dan tutorial reinforcement learning di lingkungan Taxi dari gym karena lingkungannya terinspirasi dari lingkungan tersebut. Sehingga inti dari program ini mengikuti tutorial tersebut dengan beberapa perubahan.

Berikut beberapa resource yang saya gunakan :
1. https://towardsdatascience.com/simple-reinforcement-learning-q-learning-fcddc4b6fe56
2. https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

Selebihnya dari kode kecil2 yang saya buat di notebook program merupakan buatan saya sendiri.

# Kesimpulan

Disini saya membuat program ini dengan tujuan awal mulai mempelajari ai, reinforcement learning dan dalam pengerjaannya menggunakan metode Amati, Tiru, Modifikasi. Sehingga saya merasa makalah tugas ini kurang cocok dimasukkan ke dalam jurnal dsb. Tetapi saya nanti mungkin masukkan ke dalam blog.

## Terimakasih, mohon maaf yang sebesar-besarnya jika ada kesalahan yang pernah saya perbuat